In [1]:
from google.cloud import storage
from google.cloud.exceptions import NotFound

class DataProcessor():
    
    def __init__(self, bucket_name='tbrain-tsmc', destination_blob_name='data/'):
        self.bucket_name = bucket_name
        self.destination_blob_name = destination_blob_name
        
        self.storage_client = storage.Client()
        try:
            self.storage_client.get_bucket(self.bucket_name)
        except NotFound:
             print("請聯繫Sponsor開通權限!")
                
        self.bucket = self.storage_client.bucket(self.bucket_name)
        
         
    def upload_blob(self, source_file_name, destination_file_name):
        blob = self.bucket.blob(self.destination_blob_name + destination_file_name)
        blob.upload_from_filename(source_file_name)
        print("File {} uploaded to {}.".format(source_file_name, self.destination_blob_name))
        
    def get_blob(self, source_file_name):
        blob = storage.Blob(source_file_name, self.bucket)
        content = blob.download_as_string()
        return content
        
    def delete_file(self, source_file_name):
        d_file = self.destination_blob_name + source_file_name
        try:
            self.bucket.delete_blob(d_file)
            print("File {} id deleted successfully.".format(d_file))
        except NotFound:
            print("File {} doesn't exist.".format(d_file))
        
    def show_files(self,max_results_=10, prefix_='data/YUSUN.zip/YUSUN/textFiles/'):
        all_blobs = list(self.storage_client.list_blobs(self.bucket,max_results=max_results_, prefix=prefix_))
        print(all_blobs)